In [2]:
# ▶️ 1. Google Drive 연결
from google.gdown import drive
drive.mount('/content/drive')
import os

# 폴더가 없다면 자동 생성
folder_path = '/content/drive/MyDrive/my_project'
os.makedirs(folder_path, exist_ok=True)

# ▶️ 2. 필요한 모듈
import requests
import pandas as pd
import time

# ▶️ 3. API 키 설정
tmdb_api_key = 'efc7d809310cb48568b7916e04369dbe'
omdb_api_key = 'd503f224'

# ▶️ 4. API URL 설정
tmdb_discover_url = 'https://api.themoviedb.org/3/discover/movie'
tmdb_detail_url = 'https://api.themoviedb.org/3/movie/{}'
tmdb_keywords_url = 'https://api.themoviedb.org/3/movie/{}/keywords'
omdb_url = 'http://www.omdbapi.com/'

headers = {'Accept': 'application/json'}
language = 'ko-KR'

# ▶️ 5. 결과 저장 리스트
all_movies = []

# ▶️ 6. 연도별 수집 루프
for year in range(2015, 2026):
    page = 1
    total_pages = 1

    while page <= total_pages:
        params = {
            'api_key': tmdb_api_key,
            'language': language,
            'sort_by': 'popularity.desc',
            'primary_release_year': year,
            'page': page
        }

        try:
            response = requests.get(tmdb_discover_url, headers=headers, params=params)
            data = response.json()
        except Exception as e:
            print(f"❌ TMDB 요청 실패 (연도: {year}, 페이지: {page}) → {e}")
            break

        if page == 1:
            total_pages = data.get('total_pages', 1)

        for movie in data.get('results', []):
            tmdb_id = movie.get('id')
            title = movie.get('title')
            release_date = movie.get('release_date')
            media_type = "movie"
            vote_average = movie.get('vote_average')

            # ▶️ TMDB 상세 정보 요청
            try:
                detail_resp = requests.get(tmdb_detail_url.format(tmdb_id), headers=headers,
                                           params={'api_key': tmdb_api_key, 'language': language})
                detail = detail_resp.json()
            except Exception as e:
                print(f"⚠️ TMDB 상세 실패 - {tmdb_id} : {e}")
                continue

            imdb_id = detail.get('imdb_id')
            runtime = detail.get('runtime')
            genres = ", ".join([g['name'] for g in detail.get('genres', [])])
            production = detail.get('production_companies', [])
            production_company = production[0]['name'] if production else None

            # ▶️ TMDB 키워드
            try:
                keywords_resp = requests.get(tmdb_keywords_url.format(tmdb_id), headers=headers,
                                             params={'api_key': tmdb_api_key})
                keywords_data = keywords_resp.json()
                tags = ", ".join([kw['name'] for kw in keywords_data.get('keywords', [])])
            except:
                tags = None

            # ▶️ OMDb 요청
            director = cast = country = language_omdb = plot = imdb_rating = imdb_votes = None
            if imdb_id:
                try:
                    omdb_params = {'apikey': omdb_api_key, 'i': imdb_id}
                    omdb_resp = requests.get(omdb_url, params=omdb_params)
                    omdb_data = omdb_resp.json()

                    director = omdb_data.get('Director')
                    cast = omdb_data.get('Actors')
                    country = omdb_data.get('Country')
                    language_omdb = omdb_data.get('Language')
                    plot = omdb_data.get('Plot')
                    imdb_rating = omdb_data.get('imdbRating')
                    imdb_votes = omdb_data.get('imdbVotes')
                except:
                    print(f"⚠️ OMDb 요청 실패 - imdb_id: {imdb_id}")
                    continue

            # ▶️ 결과 저장
            all_movies.append({
                'title': title,
                'release_date': release_date,
                'type': media_type,
                'runtime': runtime,
                'genre': genres,
                'director': director,
                'cast': cast,
                'country': country,
                'language': language_omdb,
                'production_company': production_company,
                'synopsis': plot,
                'tags': tags,
                'imdb_rating': imdb_rating,
                'imdb_review_count': imdb_votes,
                'imdb_id': imdb_id
            })

            time.sleep(0.3)  # 과도한 요청 방지

        print(f"[{year}] 페이지 {page}/{total_pages} 수집 완료")
        page += 1

# ▶️ 7. 데이터프레임 변환 및 Google Drive 저장
if all_movies:
    df = pd.DataFrame(all_movies)
    df.to_csv(f'{folder_path}/tmdb_imdb_combined_2015_2025.csv', index=False)
    print("✅ Google Drive 저장 완료!")
else:
    print("❌ 수집된 데이터가 없습니다.")

ModuleNotFoundError: No module named 'google.gdown'